<a href="https://colab.research.google.com/github/WellingtonLandim/wellingtonlandim.github.io/blob/main/PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=e275ba3dc4f85b717c2cee313e937d859ca73438818eae1d263dbbe897871e0b
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType, DoubleType

In [2]:
spark = SparkSession.builder.master('local[*]').getOrCreate()

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
! ls /content/drive/'My Drive'/'00 - XPE'/'Modulo 2'/'Desafio'

'Desafio Módulo 2 (2).ipynb'
 Desafio_Módulo_2_corrigido.ipynb
'Enunciado do Desafio - Módulo 2 - Bootcamp Cientista de Dados.pdf-1.pdf'
 stroke_data.csv


In [5]:
sqlc = SQLContext(spark.sparkContext)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [6]:
stroke_df = sqlc.read.csv("/content/drive/My Drive/00 - XPE/Modulo 2/Desafio/stroke_data.csv", header='True', inferSchema='True')

In [7]:
stroke_df.show()

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

In [8]:
# Pergunta 1: Quantos registros existem no arquivo?
totalRegistros = lit(stroke_df.count())
totalRegistros
# Resposta: 67135

Column<'67135'>

In [9]:
# Pergunta 2: Quantas colunas existem no arquivo? Quantas são numéricas? Ao ler o arquivo com spark.read.csv, habilite inferSchema=True. Use a função printSchema() da API de Dataframes.
# Resposta: 12 e 7
stroke_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [10]:
# Total de Colunas
totalColunas = lit(len(stroke_df.columns))
totalColunas

Column<'12'>

In [11]:
# Total de Colunas numéricas
totalColunasNumericas = 0
for coluna in stroke_df.schema:
    if isinstance(coluna.dataType, IntegerType) or isinstance(coluna.dataType, DoubleType):
        totalColunasNumericas += 1

print("Número de colunas numéricas:", totalColunasNumericas)

Número de colunas numéricas: 7


In [12]:
# Pergunta 3: No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke), respectivamente?
# Resposta: 40287 e 26848
tipoPaciente = stroke_df.groupBy("stroke").count()
tipoPaciente.show()

+------+-----+
|stroke|count|
+------+-----+
|     1|40287|
|     0|26848|
+------+-----+



In [13]:
# Pergunta 4: A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e a seguir use spark.sql para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (work_type). Quantos pacientes sofreram derrame e trabalhavam respectivamente, no setor privado, de forma independente, no governo e quantas são crianças?
# Resposta: 23711, 1080, 5164, 520
# Tabela Temporária
stroke_df.createOrReplaceTempView("stroke")


In [14]:
# Pacientes com derrame (stroke=1) por tipo de trabalho
sqlDF = spark.sql("SELECT work_type, COUNT(*) FROM stroke WHERE stroke = 1 GROUP BY work_type ")
sqlDF.show()
# Setor Privado = 23711
# Independente = 1080
# Governo = 5164
# Crianças = 520

+-------------+--------+
|    work_type|count(1)|
+-------------+--------+
| Never_worked|      85|
|Self-employed|   10807|
|      Private|   23711|
|     children|     520|
|     Govt_job|    5164|
+-------------+--------+



In [15]:
# Pergunta 5: Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de participantes do estudo. A maioria dos participantes é:
# Resposta 5: feminina
sqlGenero = spark.sql("SELECT gender, COUNT(*) as total FROM stroke GROUP BY gender")
sqlGenero.show()

+------+-----+
|gender|total|
+------+-----+
|Female|39530|
| Other|   11|
|  Male|27594|
+------+-----+



In [16]:
# Pergunta 6: Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos. Você pode escrever uma consulta para cada grupo. A partir das probabilidades que você obteve, você conclui que:
# Resposta 6: A hipertensão, neste conjunto de dados, aumenta a probabilidade de derrame.
# Teve derrame
sqlHiperStroke = spark.sql("SELECT hypertension, COUNT(*) as totalComDerrame FROM stroke WHERE stroke= 1 GROUP BY hypertension")
sqlHiperStroke.show()

+------------+---------------+
|hypertension|totalComDerrame|
+------------+---------------+
|           1|           8817|
|           0|          31470|
+------------+---------------+



In [17]:
# Não teve derrame
sqlHiperNotStroke = spark.sql("SELECT hypertension, COUNT(*) as totalSemDerrame FROM stroke WHERE stroke= 0 GROUP BY hypertension")
sqlHiperNotStroke.show()

+------------+---------------+
|hypertension|totalSemDerrame|
+------------+---------------+
|           1|           2200|
|           0|          24648|
+------------+---------------+



In [18]:
# Tabela de Contigência
tabelaContingencia = sqlHiperStroke.join(sqlHiperNotStroke, "hypertension", "outer")
tabelaContingencia.show()

+------------+---------------+---------------+
|hypertension|totalComDerrame|totalSemDerrame|
+------------+---------------+---------------+
|           0|          31470|          24648|
|           1|           8817|           2200|
+------------+---------------+---------------+



In [19]:
# Comparar as probabilidades
probHipertenso = tabelaContingencia.collect()[1]["totalComDerrame"] / (tabelaContingencia.collect()[1]["totalComDerrame"]+tabelaContingencia.collect()[1]["totalSemDerrame"])
probNaoHipertenso = tabelaContingencia.collect()[0]["totalComDerrame"] / (tabelaContingencia.collect()[0]["totalComDerrame"]+tabelaContingencia.collect()[0]["totalSemDerrame"])

print("Probabilidade de derrame para hipertensos:", probHipertenso)
print("Probabilidade de derrame para não-hipertensos:", probNaoHipertenso)


# Conclusão
# A hipertensão, neste conjunto de dados, aumenta a probabilidade de derrame.


Probabilidade de derrame para hipertensos: 0.8003086139602432
Probabilidade de derrame para não-hipertensos: 0.5607826365871913


In [20]:
# Pergunta 7: Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade. Com qual idade o maior número de pessoas do conjunto de dados sofreu derrame?
# Resposta 7: 79
tabelaIdade = spark.sql("SELECT age, COUNT(*) FROM stroke GROUP BY age ORDER BY COUNT(*) DESC")
tabelaIdade.show()

+----+--------+
| age|count(1)|
+----+--------+
|79.0|    3258|
|78.0|    2672|
|80.0|    2141|
|81.0|    2005|
|82.0|    1657|
|63.0|    1294|
|66.0|    1195|
|77.0|    1190|
|74.0|    1184|
|57.0|    1160|
|70.0|    1151|
|76.0|    1088|
|67.0|    1070|
|51.0|    1067|
|65.0|    1046|
|75.0|    1015|
|52.0|    1003|
|58.0|     999|
|59.0|     994|
|61.0|     988|
+----+--------+
only showing top 20 rows



In [21]:
# Pergunta 8: Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.
# Resposta 8: 28938
stroke_df.filter((stroke_df["age"]>50) & (stroke_df["stroke"]==1)).count()


28938

In [22]:
# Pergunta 9: Usando spark.sql, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.
# Resposta 9: 119 e 103
# Sofreram Derrame
sqlGlicoseDerrame = spark.sql("SELECT AVG(avg_glucose_level) FROM stroke WHERE stroke=1")
sqlGlicoseDerrame.show()

+----------------------+
|avg(avg_glucose_level)|
+----------------------+
|    119.95307046938272|
+----------------------+



In [23]:
# Não Sofreram Derrame
sqlGlicoseNaoDerrame = spark.sql("SELECT AVG(avg_glucose_level) FROM stroke WHERE stroke=0")
sqlGlicoseNaoDerrame.show()

+----------------------+
|avg(avg_glucose_level)|
+----------------------+
|    103.60273130214506|
+----------------------+



In [24]:
# Pergunta 10: Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?
# Resposta 10: 29,94 e 27,99
# IMC(bmi) Sofreu derrame
sqlIMCDerrame = spark.sql("SELECT AVG(bmi) FROM stroke WHERE stroke=1")
sqlIMCDerrame.show()

+------------------+
|          avg(bmi)|
+------------------+
|29.942490629729495|
+------------------+



In [25]:
# IMC(bmi) não Sofreu derrame
sqlIMCDerrame = spark.sql("SELECT AVG(bmi) FROM stroke WHERE stroke=0")
sqlIMCDerrame.show()

+------------------+
|          avg(bmi)|
+------------------+
|27.989678933253657|
+------------------+



In [26]:
# Pergunra 11: Crie um modelo de árvore de decisão que prevê a chance de derrame (stroke) a partir das variáveis contínuas/categóricas:
# idade, BMI, hipertensão, doença do coração, nível médio de glicose.
# Use o conteúdo da segunda aula interativa para criar e avaliar o modelo.
# Resposta 11: Menor que 75%
stroke_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [27]:
# Bibliotecas para ML
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer, VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [28]:
# Colunas selecionadas - todas são numericas
feature_cols = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")


In [29]:
# Instacia o DecissionTreeClassifier
dt = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')


In [30]:
# Cria o pipeline
pipeline = Pipeline(stages=[assembler, dt])

In [31]:
# Divide os dados em treino e teste
(train, test) = stroke_df.randomSplit([0.7, 0.3], seed=42)

In [32]:
# Treina o modelo
model = pipeline.fit(train)

In [33]:
# Avalia o modelo com accurácia
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='stroke')
accuracyDadosTeste = evaluator.evaluate(model.transform(test))
print("Acurácia:", accuracyDadosTeste)
# Resposta: Menor que 75%

Acurácia: 0.6884394778481012


In [34]:
# Pergunta 12: Adicione ao modelo as variáveis categóricas:
# gênero e status de fumante.
# Use o conteúdo da aula interativa para lidar com as variáveis categóricas.  A acurácia (qualidade) do modelo aumentou para:
# Resposta 12: Acima de 80%
feature_cols_cat = ['gender', 'smoking_status']


In [35]:
# Analisar os dados
# gender
gender = stroke_df.groupBy("gender").count()
gender.show()

+------+-----+
|gender|count|
+------+-----+
|Female|39530|
| Other|   11|
|  Male|27594|
+------+-----+



In [36]:
# smoking_status
smoking_status = stroke_df.groupBy("smoking_status").count()
smoking_status.show()

+---------------+-----+
| smoking_status|count|
+---------------+-----+
|         smokes|35963|
|   never_smoked|13292|
|formerly smoked|17880|
+---------------+-----+



In [37]:
# Transformar colunas
from pyspark.ml.feature import StringIndexer, OneHotEncoder


In [38]:
# Lista de colunas categóricas e numéricas
categoricalCols = ['gender', 'smoking_status']
numericCols = ["age", "bmi", "hypertension", "heart_disease", "avg_glucose_level"]

# Transformar colunas categóricas com StringIndexer
indexers = [StringIndexer(inputCol=col, outputCol=col+"_index", handleInvalid="keep") for col in categoricalCols]

# Codificar colunas indexadas one-hot
encoders = [OneHotEncoder(inputCol=col+"_index", outputCol=col+"_OHE") for col in categoricalCols]

# Combina todas as colunas em uma única coluna vetor
assemblerInputs = [col + "_OHE" for col in categoricalCols] + numericCols
vecAssembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# Instanciar o DecisionTreeClassifier
dt = DecisionTreeClassifier(labelCol='stroke', featuresCol='features')

# Criar o pipeline
pipeline = Pipeline(stages=indexers + encoders + [vecAssembler, dt])

# Dividir os dados em conjunto de treinamento e teste
(train, test) = stroke_df.randomSplit([0.7, 0.3])

# Treinar o modelo
model = pipeline.fit(train)

In [39]:
# Avalia o modelo com accurácia com todas as colunas
evaluator = MulticlassClassificationEvaluator(metricName='accuracy', labelCol='stroke')
accuracyDadosTeste = evaluator.evaluate(model.transform(test))
print("Acurácia:", accuracyDadosTeste)
# Resposta 12: Acima de 80%

Acurácia: 0.8369661702337509


In [40]:
# Pergunta 13: Adicione ao modelo as variáveis categóricas: gênero e status de fumante. Use o conteúdo da aula interativa para lidar com as variáveis categóricas. Qual dessas variáveis é mais importante no modelo de árvore de decisão que você construiu?
# Resposta 13:  avg_glucose_level. ERRADO

# Obter importâncias das características do modelo treinado
feature_importances = model.stages[-1].featureImportances

# Criar um dicionário para mapear os nomes das características para suas importâncias
feature_importance_dict = dict(zip(numericCols + feature_cols_cat, feature_importances))

# Classificar as características com base em suas importâncias e imprimir
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
for feature, importance in sorted_features:
    print(f"Variável: {feature}, Importância: {importance}")

Variável: gender, Importância: 0.7359929060005994
Variável: smoking_status, Importância: 0.17224764563488046
Variável: heart_disease, Importância: 0.07996401200523502
Variável: age, Importância: 0.0
Variável: bmi, Importância: 0.0
Variável: hypertension, Importância: 0.0
Variável: avg_glucose_level, Importância: 0.0


In [41]:
# Pergunta 14: Qual a profundidade da árvore de decisão?
# Resposta 14 : depth=5. Entre 2 e 7.
# Pergunta 15: Quantos nodos a árvore de decisão possui?
# Resposta 15 : numNodes=21. Mais de 5.
print(model.stages)

[StringIndexerModel: uid=StringIndexer_8d7a5a0172d6, handleInvalid=keep, StringIndexerModel: uid=StringIndexer_00d3365961db, handleInvalid=keep, OneHotEncoderModel: uid=OneHotEncoder_9910bf27adb5, dropLast=true, handleInvalid=error, OneHotEncoderModel: uid=OneHotEncoder_46e52397b1ad, dropLast=true, handleInvalid=error, VectorAssembler_51d0569f87f1, DecisionTreeClassificationModel: uid=DecisionTreeClassifier_d6c2f428b6f0, depth=5, numNodes=21, numClasses=2, numFeatures=11]
